排放数据

In [53]:
import pandas as pd

# 读取 Excel 文件
excel_file = "emissions.xlsx"

# 获取所有 sheet 名字
sheet_names = pd.ExcelFile(excel_file).sheet_names

# 依次读取并打印每个 sheet 的名字和内容
for sheet in sheet_names:
    print(f"Sheet name: {sheet}")
    df = pd.read_excel(excel_file, sheet_name=sheet)
    print(df)
    print("\n" + "="*40 + "\n")


Sheet name: FE_emission
    Unnamed: 0      ANHU      BEIJ      CHON      FUJI      GANS      GUAD  \
0         2020  2.013547  0.587034  1.117267  1.486283  0.853643  3.276580   
1         2021  1.984903  0.612617  1.120212  1.529369  0.853823  3.081785   
2         2022  1.924324  0.520964  1.042890  1.689208  0.869817  2.969463   
3         2023  2.137066  0.613015  1.176093  1.636356  0.908834  3.249238   
4         2024  2.160842  0.609427  1.190095  1.653611  0.915279  3.280373   
5         2025  2.164423  0.606331  1.196871  1.659610  0.913209  3.300155   
6         2026  2.173344  0.603496  1.205937  1.666617  0.913268  3.319930   
7         2027  2.182766  0.599683  1.213920  1.673990  0.913223  3.340724   
8         2028  2.193898  0.595946  1.223213  1.682448  0.913737  3.366026   
9         2029  2.184099  0.575689  1.210692  1.668616  0.904992  3.327674   
10        2030  2.175231  0.555846  1.198756  1.655470  0.896681  3.291704   
11        2031  2.110216  0.531024  1.16

In [54]:
import pandas as pd
import json
import os

# --- 1. 配置 ---
excel_file = "emissions.xlsx"
output_dir = "json"
output_json_file = os.path.join(output_dir, "emissions.json")

# Mapeo de los nombres de las hojas a las claves que queremos en el JSON
sheet_map = {
    'FE_emission': 'FE',
    'Emi_supply': 'SUPPLY',
    'Emi_total': 'TOTAL'
}

# Definir los años de interés: de 2025 a 2060, en pasos de 5 años.
years_to_keep = list(range(2025, 2061, 5))

# --- 2. LECTURA Y PREPARACIÓN DE DATOS ---

# Cargar cada hoja en un DataFrame de pandas
try:
    df_fe = pd.read_excel(excel_file, sheet_name='FE_emission', index_col=0)
    df_supply = pd.read_excel(excel_file, sheet_name='Emi_supply', index_col=0)
    df_total = pd.read_excel(excel_file, sheet_name='Emi_total', index_col=0)
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{excel_file}'. Asegúrate de que esté en el mismo directorio.")
    exit()

# Crear un diccionario para acceder fácilmente a los dataframes
dataframes = {
    'FE': df_fe,
    'SUPPLY': df_supply,
    'TOTAL': df_total
}

# Obtener una lista consistente de todas las provincias (columnas)
provinces = list(set(df_fe.columns) & set(df_supply.columns) & set(df_total.columns))
provinces.sort() # Ordenar alfabéticamente para un resultado consistente

# --- 3. CONSTRUCCIÓN DE LA ESTRUCTURA JSON ---

final_data = {}

for province in provinces:
    final_data[province] = {}
    for key, df in dataframes.items():
        # Filtrar el DataFrame por los años de interés y seleccionar la columna de la provincia
        # Redondear a 3 decimales
        province_year_data = df.loc[years_to_keep, province].round(3).to_dict()
        # 转为 float 并保留 3 位小数（即使是整数也会变成 3 位小数）
        province_year_data = {str(year): float(f"{value:.3f}") if pd.notnull(value) else None for year, value in province_year_data.items()}
        final_data[province][key] = province_year_data

# --- 4. ESCRITURA DEL ARCHIVO JSON ---
os.makedirs(output_dir, exist_ok=True)
with open(output_json_file, 'w', encoding='utf-8') as f:
    json.dump(final_data, f, ensure_ascii=False, indent=4)

print(f"¡Proceso completado! Los datos han sido guardados en el archivo: '{output_json_file}'")

# Opcional: Imprimir una muestra para verificar la estructura
print("\n--- Muestra de los datos para la provincia 'ANHU' ---")
print(json.dumps({'ANHU': final_data.get('ANHU', {})}, indent=4, ensure_ascii=False))

¡Proceso completado! Los datos han sido guardados en el archivo: 'json/emissions.json'

--- Muestra de los datos para la provincia 'ANHU' ---
{
    "ANHU": {
        "FE": {
            "2025": 2.164,
            "2030": 2.175,
            "2035": 1.882,
            "2040": 1.398,
            "2045": 1.021,
            "2050": 0.596,
            "2055": 0.275,
            "2060": 0.178
        },
        "SUPPLY": {
            "2025": 2.446,
            "2030": 3.159,
            "2035": 2.879,
            "2040": 2.152,
            "2045": 1.601,
            "2050": 0.501,
            "2055": 0.501,
            "2060": -0.209
        },
        "TOTAL": {
            "2025": 4.61,
            "2030": 5.334,
            "2035": 4.76,
            "2040": 3.549,
            "2045": 2.622,
            "2050": 1.096,
            "2055": 0.776,
            "2060": -0.03
        }
    }
}


In [55]:
import pandas as pd
import json
import os

# 读取当前目录下的 "source data 对接版.xlsx" 文件中的 "资源潜力" sheet
try:
    df_resource_potential = pd.read_excel("source data 对接版.xlsx", sheet_name="资源潜力")
    
    # 创建输出数据结构
    resource_data = {}
    
    # 遍历每一行数据，提取省份和资源数据
    for index, row in df_resource_potential.iterrows():
        # 跳过最后两行说明文字
        if pd.isna(row['Unnamed: 0']) or index > 29:
            continue
            
        province_code = row['Unnamed: 0']  # 省份代码
        
        # 创建省份数据结构
        resource_data[province_code] = {
            "coal": {
                "2025": round(float(row['coal']) / 1000, 2),
                "2030": round(float(row['coal']) / 1000, 2),
                "2035": round(float(row['coal']) / 1000, 2),
                "2040": round(float(row['coal']) / 1000, 2),
                "2045": round(float(row['coal']) / 1000, 2),
                "2050": round(float(row['coal']) / 1000, 2),
                "2055": round(float(row['coal']) / 1000, 2),
                "2060": round(float(row['coal']) / 1000, 2)
            },
            "oil": {
                "2025": round(float(row['oil']), 2),
                "2030": round(float(row['oil']), 2),
                "2035": round(float(row['oil']), 2),
                "2040": round(float(row['oil']), 2),
                "2045": round(float(row['oil']), 2),
                "2050": round(float(row['oil']), 2),
                "2055": round(float(row['oil']), 2),
                "2060": round(float(row['oil']), 2)
            },
            "gas": {
                "2025": round(float(row['gas']), 2),
                "2030": round(float(row['gas']), 2),
                "2035": round(float(row['gas']), 2),
                "2040": round(float(row['gas']), 2),
                "2045": round(float(row['gas']), 2),
                "2050": round(float(row['gas']), 2),
                "2055": round(float(row['gas']), 2),
                "2060": round(float(row['gas']), 2)
            },
            "nuclear": {
                "2025": round(float(row['nuclear']), 2),
                "2030": round(float(row['nuclear']), 2),
                "2035": round(float(row['nuclear']), 2),
                "2040": round(float(row['nuclear']), 2),
                "2045": round(float(row['nuclear']), 2),
                "2050": round(float(row['nuclear']), 2),
                "2055": round(float(row['nuclear']), 2),
                "2060": round(float(row['nuclear']), 2)
            },
            "biomass": {
                "2025": round(float(row['biomass']), 2),
                "2030": round(float(row['biomass']), 2),
                "2035": round(float(row['biomass']), 2),
                "2040": round(float(row['biomass']), 2),
                "2045": round(float(row['biomass']), 2),
                "2050": round(float(row['biomass']), 2),
                "2055": round(float(row['biomass']), 2),
                "2060": round(float(row['biomass']), 2)
            },
            "hydro": {
                "2025": round(float(row['hydro']), 2),
                "2030": round(float(row['hydro']), 2),
                "2035": round(float(row['hydro']), 2),
                "2040": round(float(row['hydro']), 2),
                "2045": round(float(row['hydro']), 2),
                "2050": round(float(row['hydro']), 2),
                "2055": round(float(row['hydro']), 2),
                "2060": round(float(row['hydro']), 2)
            },
            "wind": {
                "2025": round(float(row['onwind']) + float(row['offwind']), 2),
                "2030": round(float(row['onwind']) + float(row['offwind']), 2),
                "2035": round(float(row['onwind']) + float(row['offwind']), 2),
                "2040": round(float(row['onwind']) + float(row['offwind']), 2),
                "2045": round(float(row['onwind']) + float(row['offwind']), 2),
                "2050": round(float(row['onwind']) + float(row['offwind']), 2),
                "2055": round(float(row['onwind']) + float(row['offwind']), 2),
                "2060": round(float(row['onwind']) + float(row['offwind']), 2)
            },
            "solar": {
                "2025": round(float(row['pv']), 2),
                "2030": round(float(row['pv']), 2),
                "2035": round(float(row['pv']), 2),
                "2040": round(float(row['pv']), 2),
                "2045": round(float(row['pv']), 2),
                "2050": round(float(row['pv']), 2),
                "2055": round(float(row['pv']), 2),
                "2060": round(float(row['pv']), 2)
            }
        }
    
    # 创建json目录（如果不存在）
    os.makedirs('json', exist_ok=True)
    
    # 将数据保存到 json/resource.json 文件
    with open('json/resource.json', 'w', encoding='utf-8') as f:
        json.dump(resource_data, f, ensure_ascii=False, indent=4)
    
    print("资源数据已成功保存到 json/resource.json 文件")
    
except FileNotFoundError:
    print("未找到文件 'source data 对接版.xlsx'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理资源潜力数据时发生错误: {e}")


资源数据已成功保存到 json/resource.json 文件


读取电量调入调出

In [18]:
# 读取电量调入调出数据
try:
    import json
    import os
    
    elc_net_in_file = 'ELC net in.xlsx'
    elc_net_in_df = pd.read_excel(elc_net_in_file)

    # 打印完整数据
    print("电量调入调出数据:")
    print(elc_net_in_df)

    # 显示数据基本信息
    print("\n数据基本信息:")
    print(f"行数: {elc_net_in_df.shape[0]}")
    print(f"列数: {elc_net_in_df.shape[1]}")
    print("\n列名:")
    print(elc_net_in_df.columns.tolist())
    
    # 创建电量调入调出的JSON数据结构
    elc_trans_data = {}
    
    # 遍历每一行数据（每个省份）
    for _, row in elc_net_in_df.iterrows():
        province_code = row['Unnamed: 0']  # 获取省份代码
        
        # 为每个省份创建数据结构
        elc_trans_data[province_code] = {
            "ELC_TRA": {
                "2020": round(float(row['2020 net']), 1),
                "2025": round(float(row['2025 net']), 1),
                "2030": round(float(row['2030 net']), 1),
                "2035": round(float(row['2035 net']), 1),
                "2040": round(float(row['2040 net']), 1),
                "2045": round(float(row['2045 net']), 1),
                "2050": round(float(row['2050 net']), 1),
                "2055": round(float(row['2055 net']), 1),
                "2060": round(float(row['2060 net']), 1)
            }
        }
    
    # 确保json目录存在
    os.makedirs('json', exist_ok=True)
    
    # 将数据保存到 json/elc_trans.json 文件
    with open('json/elc_trans.json', 'w', encoding='utf-8') as f:
        json.dump(elc_trans_data, f, ensure_ascii=False, indent=4)
    
    print("\n电量调入调出数据已成功保存到 json/elc_trans.json 文件")
    
except FileNotFoundError:
    print(f"未找到文件 '{elc_net_in_file}'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理电量调入调出数据时发生错误: {e}")


电量调入调出数据:
   Unnamed: 0    2020 net    2025 net    2030 net    2035 net    2040 net  \
0        BEIJ   99.579710  126.069502  162.292979  205.312392  246.684709   
1        TIAN   25.473061   33.992648   47.154909   66.310141   94.276287   
2        HEBE   80.776476   90.574618  100.288093  100.017456  106.673988   
3        SHNX  -91.984864  -82.019460  -92.120157 -103.578345 -114.490321   
4        NEMO -222.715017 -324.793360 -436.865559 -556.576536 -742.615823   
5        LIAO   48.532156   49.880787   36.890045   -7.587407  -54.409824   
6        JILI  -27.071590  -22.022940  -14.176866  -23.842955  -31.097525   
7        HEIL    3.282850    6.503406    8.679614   14.229045   20.041528   
8        SHAN  124.018095  173.729204  246.062140  302.198119  351.164657   
9        JINU   70.786935   76.549132   69.365755  135.509203  198.094371   
10       ZHEJ  127.424543  158.059852   98.328640   12.061403  -49.500793   
11       ANHU  -25.253073  -71.246142  -58.213285  -56.129212  -37

读取氢能的制备

In [57]:
try:
    # 定义氢能制备技术文件路径
    h2n_pro_tech_file = 'H2N_pro_tech.xlsx'
    
    # 读取氢能制备技术数据
    h2n_pro_tech_df = pd.read_excel(h2n_pro_tech_file)
    
    # 打印整个表格
    print("氢能制备技术数据:")
    print(h2n_pro_tech_df.to_string())
    
    # 打印表格的基本信息
    print("\n表格信息:")
    print(f"行数: {h2n_pro_tech_df.shape[0]}")
    print(f"列数: {h2n_pro_tech_df.shape[1]}")
    print("\n列名:")
    print(h2n_pro_tech_df.columns.tolist())
    
    # 创建氢能制备数据的JSON结构
    h2n_data = {}
    
    # 初始化省份列表
    provinces = []
    for i in range(1, 270):
        if not pd.isna(h2n_pro_tech_df.iloc[i, 1]):  # 'Unnamed: 1'列不为空
            province = h2n_pro_tech_df.iloc[i, 1]
            if province not in provinces:
                provinces.append(province)
                h2n_data[province] = {
                    "ELC": {},      # 电解水制氢
                    "solar": {},    # 太阳能制氢
                    "onshore": {},  # 陆上风电制氢
                    "offshore": {}  # 海上风电制氢
                }
    
    # 年份列表
    years = []
    for i in range(0, 270, 30):
        if not pd.isna(h2n_pro_tech_df.iloc[i, 0]):  # 'Unnamed: 0'列不为空
            year = int(h2n_pro_tech_df.iloc[i, 0])
            years.append(year)
    
    # 填充数据
    for year_idx, year in enumerate(years):
        start_row = year_idx * 30
        for i in range(30):
            if start_row + i >= len(h2n_pro_tech_df):
                break
            
            row = h2n_pro_tech_df.iloc[start_row + i]
            if not pd.isna(row['Unnamed: 1']):  # 确保省份不为空
                province = row['Unnamed: 1']
                year_str = str(year)
                
                # 直接读取各列的值
                elc_value = row['ELC']
                solar_value = row['solar']
                onshore_value = row['onshore']
                offshore_value = row['offshore']
                
                # 添加到对应省份的数据结构中，保留一位小数
                if not pd.isna(elc_value):
                    h2n_data[province]["ELC"][year_str] = round(float(elc_value), 1)
                if not pd.isna(solar_value):
                    h2n_data[province]["solar"][year_str] = round(float(solar_value), 1)
                if not pd.isna(onshore_value):
                    h2n_data[province]["onshore"][year_str] = round(float(onshore_value), 1)
                if not pd.isna(offshore_value):
                    h2n_data[province]["offshore"][year_str] = round(float(offshore_value), 1)
    
    # 确保json目录存在
    import os
    if not os.path.exists('json'):
        os.makedirs('json')
    
    # 将数据保存到 json/h2n.json 文件
    with open('json/h2n.json', 'w', encoding='utf-8') as f:
        json.dump(h2n_data, f, ensure_ascii=False, indent=4)
    
    print("\n氢能制备数据已成功保存到 json/h2n.json 文件")
    
except FileNotFoundError:
    print(f"未找到文件 '{h2n_pro_tech_file}'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理氢能制备技术数据时发生错误: {e}")


氢能制备技术数据:
     Unnamed: 0 Unnamed: 1         ELC       solar     onshore    offshore
0        2020.0       BEIJ    0.000000    0.000000    0.000000    0.000000
1           NaN       TIAN    0.000000    0.000000    0.000000    0.000000
2           NaN       HEBE    0.000000    0.000000    0.000000    0.000000
3           NaN       SHNX    0.000000    0.000000    0.000000    0.000000
4           NaN       NEMO    0.000000    0.000000    0.000000    0.000000
5           NaN       LIAO    0.000000    0.000000    0.000000    0.000000
6           NaN       JILI    0.000000    0.000000    0.000000    0.000000
7           NaN       HEIL    0.000000    0.000000    0.000000    0.000000
8           NaN       SHAN    0.000000    0.000000    0.000000    0.000000
9           NaN       JINU    0.000000    0.000000    0.000000    0.000000
10          NaN       ZHEJ    0.000000    0.000000    0.000000    0.000000
11          NaN       ANHU    0.000000    0.000000    0.000000    0.000000
12          NaN

一次能源

In [58]:
# 读取一次能源数据
try:
    pe_file = 'PE_pro_fuel.xlsx'
    
    # 读取Excel文件
    pe_df = pd.read_excel(pe_file)
    
    # 创建数据结构
    pe_data = {}
    
    # 遍历DataFrame处理数据
    for _, row in pe_df.iterrows():
        province = row['Province']
        year = str(int(row['Year']))
        
        # 确保省份存在于数据结构中
        if province not in pe_data:
            pe_data[province] = {
                'Coal': {}, 'Coal CCS': {}, 'Oil': {}, 'Oil CCS': {},
                'Gas': {}, 'Gas CCS': {}, 'Nuclear': {}, 'Hydro': {},
                'Biomass': {}, 'Biomass CCS': {}, 'Wind': {}, 'PV': {}
            }
        
        # 添加各种能源数据
        for fuel_type in ['Coal', 'Coal CCS', 'Oil', 'Oil CCS', 'Gas', 'Gas CCS', 
                          'Nuclear', 'Hydro', 'Biomass', 'Biomass CCS', 'Wind', 'PV']:
            if not pd.isna(row[fuel_type]):
                pe_data[province][fuel_type][year] = round(float(row[fuel_type]), 1)
    
    # 确保json目录存在
    import os
    if not os.path.exists('json'):
        os.makedirs('json')
    
    # 将数据保存到 json/pe.json 文件
    with open('json/pe.json', 'w', encoding='utf-8') as f:
        json.dump(pe_data, f, ensure_ascii=False, indent=4)
    
    print("\n一次能源数据已成功保存到 json/pe.json 文件")
    
except FileNotFoundError:
    print(f"未找到文件 '{pe_file}'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理一次能源数据时发生错误: {e}")



一次能源数据已成功保存到 json/pe.json 文件


电力新增装机

In [59]:
try:
    # 定义文件路径
    capnew_file = 'capnew_pro_tech.xlsx'
    
    # 读取Excel文件
    capnew_df = pd.read_excel(capnew_file)
    
    # 创建数据结构
    capnew_data = {}
    
    # 遍历DataFrame处理数据
    current_year = None
    
    for _, row in capnew_df.iterrows():
        # 获取年份和省份
        if not pd.isna(row['Unnamed: 0']):
            current_year = str(int(row['Unnamed: 0']))
        
        province = row['Unnamed: 1']
        
        # 确保省份存在于数据结构中
        if province not in capnew_data:
            capnew_data[province] = {
                'coal': {}, 'coal ccs': {}, 'oil': {}, 'gas': {}, 'gas ccs': {},
                'nuclear': {}, 'hydro': {}, 'biomass': {}, 'biomass ccs': {}, 
                'co-firing beccs': {}, 'wind': {}, 'pv': {}
            }
        
        # 添加各种电力技术数据
        for tech in ['coal', 'coal ccs', 'oil', 'gas', 'gas ccs', 'nuclear', 'hydro', 
                     'biomass', 'biomass ccs', 'co-firing beccs', 'wind', 'pv']:
            if not pd.isna(row[tech]):
                capnew_data[province][tech][current_year] = round(float(row[tech]), 1)
    
    # 确保json目录存在
    import os
    if not os.path.exists('json'):
        os.makedirs('json')
    
    # 将数据保存到 json/newcap.json 文件
    with open('json/newcap.json', 'w', encoding='utf-8') as f:
        json.dump(capnew_data, f, ensure_ascii=False, indent=4)
    
    print("\n电力新增装机数据已成功保存到 json/newcap.json 文件")
    
except FileNotFoundError:
    print(f"未找到文件 '{capnew_file}'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理电力新增装机数据时发生错误: {e}")



电力新增装机数据已成功保存到 json/newcap.json 文件


读取 电力总装机

In [60]:
try:
    # 定义文件路径
    cap_file = 'cap_pro_tech.xlsx'
    
    # 读取Excel文件
    cap_df = pd.read_excel(cap_file)
    
    # 创建数据结构
    cap_data = {}
    
    # 遍历DataFrame处理数据
    current_year = None
    
    for _, row in cap_df.iterrows():
        # 获取年份和省份
        if not pd.isna(row['Unnamed: 0']):
            current_year = str(int(row['Unnamed: 0']))
        
        province = row['Unnamed: 1']
        
        # 确保省份存在于数据结构中
        if province not in cap_data:
            cap_data[province] = {
                'coal': {}, 'coal ccs': {}, 'oil': {}, 'gas': {}, 'gas ccs': {},
                'nuclear': {}, 'hydro': {}, 'biomass': {}, 'biomass ccs': {}, 
                'co-firing beccs': {}, 'wind': {}, 'pv': {}
            }
        
        # 添加各种电力技术数据
        for tech in ['coal', 'coal ccs', 'oil', 'gas', 'gas ccs', 'nuclear', 'hydro', 
                     'biomass', 'biomass ccs', 'co-firing beccs', 'wind', 'pv']:
            if not pd.isna(row[tech]):
                cap_data[province][tech][current_year] = round(float(row[tech]), 1)
    
    # 确保json目录存在
    import os
    if not os.path.exists('json'):
        os.makedirs('json')
    
    # 将数据保存到 json/cap.json 文件
    with open('json/cap.json', 'w', encoding='utf-8') as f:
        json.dump(cap_data, f, ensure_ascii=False, indent=4)
    
    print("\n电力总装机数据已成功保存到 json/cap.json 文件")
    
except FileNotFoundError:
    print(f"未找到文件 '{cap_file}'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理电力总装机数据时发生错误: {e}")



电力总装机数据已成功保存到 json/cap.json 文件


发电结构

In [61]:
# 读取发电结构数据
try:
    # 定义文件路径
    elec_pro_tech_file = 'elec_pro_tech.xlsx'
    
    # 读取Excel文件中的所有sheet
    elec_pro_tech_sheets = pd.read_excel(elec_pro_tech_file, sheet_name=None)
    
    # 打印所有sheet的内容
    print(f"\n读取 {elec_pro_tech_file} 文件中的所有sheet内容:")
    
    for sheet_name, sheet_data in elec_pro_tech_sheets.items():
        print(f"\n\n===== Sheet: {sheet_name} =====")
        
        # 设置显示选项，确保不省略任何内容
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
            print(sheet_data)
    
    # 创建数据结构来存储发电结构数据
    elc_mix_data = {}
    
    # 处理Sheet1中的数据
    sheet_data = elec_pro_tech_sheets['Sheet1']
    current_year = None
    
    for _, row in sheet_data.iterrows():
        # 获取年份和省份
        if not pd.isna(row['Unnamed: 0']):
            current_year = str(int(row['Unnamed: 0']))
        
        province = row['Unnamed: 1']
        
        # 确保省份存在于数据结构中
        if province not in elc_mix_data:
            elc_mix_data[province] = {
                'coal': {}, 'coal ccs': {}, 'oil': {}, 'gas': {}, 'gas ccs': {},
                'nuclear': {}, 'hydro': {}, 'biomass': {}, 'biomass ccs': {}, 
                'co-firing beccs': {}, 'wind': {}, 'pv': {}
            }
        
        # 添加各种发电技术数据
        for tech in ['coal', 'coal ccs', 'oil', 'gas', 'gas ccs', 'nuclear', 'hydro', 
                     'biomass', 'biomass ccs', 'co-firing beccs', 'wind', 'pv']:
            if not pd.isna(row[tech]):
                elc_mix_data[province][tech][current_year] = round(float(row[tech]), 1)
    
    # 确保json目录存在
    import os
    if not os.path.exists('json'):
        os.makedirs('json')
    
    # 将数据保存到 json/elc_mix.json 文件
    with open('json/elc_mix.json', 'w', encoding='utf-8') as f:
        json.dump(elc_mix_data, f, ensure_ascii=False, indent=4)
    
    print("\n发电结构数据已成功保存到 json/elc_mix.json 文件")
    
except FileNotFoundError:
    print(f"未找到文件 '{elec_pro_tech_file}'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理发电结构数据时发生错误: {e}")



读取 elec_pro_tech.xlsx 文件中的所有sheet内容:


===== Sheet: Sheet1 =====
     Unnamed: 0 Unnamed: 1         coal     coal ccs      oil         gas  \
0        2020.0       BEIJ     3.600000     0.000000  1.44000  142.920000   
1           NaN       TIAN   184.680000     0.000000  0.00000   48.240000   
2           NaN       HEBE   800.640000     0.000000  0.00000    2.880000   
3           NaN       SHNX   979.920000     0.000000  0.00000   38.880000   
4           NaN       NEMO  1683.720000     0.000000  0.00000   10.224000   
5           NaN       LIAO   482.400000     0.000000  1.44000    1.080000   
6           NaN       JILI   243.720000     0.000000  0.00000    0.036000   
7           NaN       HEIL   293.400000     0.000000  0.00000    1.080000   
8           NaN       SHAN   222.840000     0.000000  0.36000   54.720000   
9           NaN       JINU  1285.200000     0.000000  0.00000  171.720000   
10          NaN       ZHEJ   756.720000     0.000000  0.01080   58.680000   
11        

分省区的煤、油、气开采量

In [2]:
# 读取"开采-调入-调出"的sheet数据
try:
    import pandas as pd
    import json
    import os
    
    # 定义文件路径
    resource_file = 'source data 对接版.xlsx'
    
    # 读取文件中的"开采-调入-调出"sheet
    extraction_data = pd.read_excel(resource_file, sheet_name='开采-调入-调出')
    
    # 创建用于存储数据的字典
    fossil_data = {}
    
    # 遍历数据框中的每一行
    for index, row in extraction_data.iterrows():
        province = row['Unnamed: 0']
        
        # 为每个省份创建数据结构
        fossil_data[province] = {
            "coal": {
                "extraction": round(float(row['coal-extract']), 1),
                "import": round(float(row['coal-in']), 1),
                "export": round(float(row['coal-out']), 1)
            },
            "oil": {
                "extraction": round(float(row['oil-extract']), 1),
                "import": round(float(row['oil-in']), 1),
                "export": round(float(row['oil-out']), 1)
            },
            "gas": {
                "extraction": round(float(row['gas-extract']), 1),
                "import": round(float(row['gas-in']), 1),
                "export": round(float(row['gas-out']), 1)
            }
        }
    
    # 确保目标目录存在
    json_dir = 'json'
    if not os.path.exists(json_dir):
        os.makedirs(json_dir)
    
    # 将数据保存为json文件
    json_file_path = os.path.join(json_dir, '2020_pe_fossil.json')
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(fossil_data, f, ensure_ascii=False, indent=4)
    
    print(f"\n煤、油、气开采-调入-调出数据已成功保存到 {json_file_path}")
    
except FileNotFoundError:
    print(f"未找到文件 '{resource_file}'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理煤、油、气开采量数据时发生错误: {e}")



煤、油、气开采-调入-调出数据已成功保存到 json/2020_pe_fossil.json


读取电网矩阵

In [27]:
import pandas as pd
import json
import numpy as np

file_path = '/Users/kevinding/Desktop/Github/energy-supply-v2/data/excel/source data 对接版.xlsx'
sheet_name = '电网矩阵'
json_path = '/Users/kevinding/Desktop/Github/energy-supply-v2/data/excel/json/elc_matrix.json'

try:
    # 读取数据
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    # 需要保存的年份
    years = ['2020', '2030', '2040', '2050', '2060']
    # 每个年份的起始行号
    year_start_rows = {
        '2020': 0,
        '2030': 33,
        '2040': 66,
        '2050': 99,
        '2060': 132
    }
    # 每个年份的省份名在第几行
    province_row_offset = 0  # 省份名在起始行
    data_row_start_offset = 1  # 数据从起始行+1开始
    n_province = 30  # 省份数量

    elc_matrix = {}

    for year in years:
        start_row = year_start_rows[year]
        # 省份名
        provinces = df.iloc[start_row + province_row_offset, 1:1 + n_province].tolist()
        # 去除空值和nan
        provinces = [str(p).strip() for p in provinces if pd.notna(p)]
        # 读取矩阵数据
        matrix = df.iloc[start_row + data_row_start_offset : start_row + data_row_start_offset + n_province, 0:1 + n_province]
        matrix = matrix.reset_index(drop=True)
        # 行省份
        row_provinces = matrix.iloc[:,0].tolist()
        row_provinces = [str(p).strip() for p in row_provinces if pd.notna(p)]
        # 只保留有效的行
        matrix = matrix.iloc[:len(provinces), 1:1 + len(provinces)]
        # 构建嵌套字典
        elc_matrix[year] = {}
        for i, from_prov in enumerate(provinces):
            elc_matrix[year][from_prov] = {}
            for j, to_prov in enumerate(provinces):
                val = matrix.iloc[i, j]
                # 处理无效值
                if isinstance(val, str) and ('#' in val or val.strip() == ''):
                    elc_matrix[year][from_prov][to_prov] = None
                elif pd.isna(val):
                    elc_matrix[year][from_prov][to_prov] = None
                else:
                    try:
                        elc_matrix[year][from_prov][to_prov] = float(val)
                    except:
                        elc_matrix[year][from_prov][to_prov] = None

    # 保存为json
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(elc_matrix, f, ensure_ascii=False, indent=4)
    print(f"电力传输矩阵已保存到 {json_path}")

except FileNotFoundError:
    print(f"未找到文件 '{file_path}'，请确保文件路径正确。")
except Exception as e:
    print(f"读取电网矩阵时发生错误: {e}")


电力传输矩阵已保存到 /Users/kevinding/Desktop/Github/energy-supply-v2/data/excel/json/elc_matrix.json


读取 excel 的 电力投资

In [4]:
# 解析并导出电力投资数据为JSON
import pandas as pd
import json
import os
import numpy as np

try:
    # 读取Excel文件中的"电力投资"sheet
    file_path = "/Users/kevinding/Desktop/Github/energy-supply-v2/data/excel/source data 对接版.xlsx"
    df_investment = pd.read_excel(file_path, sheet_name="电力投资")
    
    # 设置显示选项
    pd.set_option('display.max_rows', 50)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    print("==== 电力投资数据解析与导出 ====")
    
    # 定义技术列表
    tech_cols = ['coal', 'oil', 'gas', 'hydro', 'offwind', 'onwind', 'pv', 'nuclear', 'biomass', 'coalccs', 'gasccs', 'beccs']
    
    # 定义年份列表
    years = [2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
    
    # 创建结果字典
    result = {}
    
    # 处理2025年的特殊情况（在文件的开头部分）
    # 找到2025年的数据块
    provinces_2025 = []
    # 2025年的数据在文件开头，没有单独的年份行，省份直接从第一行开始
    for idx, row in df_investment.iterrows():
        first_col_val = row.iloc[0]
        # 如果碰到年份行，说明2025的数据块结束了
        if pd.notna(first_col_val) and first_col_val in years and first_col_val != 2025:
            break
        # 如果是省份代码
        if isinstance(first_col_val, str) and first_col_val not in tech_cols:
            provinces_2025.append(idx)
    
    # 处理每个年份块
    for year in years:
        # 查找年份行的索引
        year_idx = df_investment.index[df_investment.iloc[:, 0] == year].tolist()
        
        if not year_idx:
            if year == 2025:  # 2025年特殊处理，因为没有单独的年份行
                start_idx = 0
            else:
                print(f"未找到{year}年的数据")
                continue
        else:
            start_idx = year_idx[0]
        
        # 查找下一个年份的索引，确定当前年份块的范围
        next_year_idx = None
        next_years = [y for y in years if y > year]
        if next_years:
            next_year = next_years[0]
            next_idx = df_investment.index[df_investment.iloc[:, 0] == next_year].tolist()
            if next_idx:
                next_year_idx = next_idx[0]
        
        # 确定当前年份块的结束索引
        end_idx = next_year_idx if next_year_idx else len(df_investment)
        
        # 获取当前年份块中的所有行
        if year == 2025:
            year_block = df_investment.iloc[provinces_2025]
        else:
            # 跳过年份行（年份行的索引是start_idx），从start_idx+1开始
            year_block = df_investment.iloc[start_idx+1:end_idx]
        
        # 遍历年份块中的每一行（每个省份）
        for _, row in year_block.iterrows():
            province = row.iloc[0]
            # 跳过非省份行和空行
            if not isinstance(province, str) or pd.isna(province) or province in tech_cols or province in [str(y) for y in years]:
                continue
            
            # 标准化省份名称
            province_key = 'national' if province == 'China' else province
            
            # 确保省份在结果字典中
            if province_key not in result:
                result[province_key] = {}
            
            # 添加每个技术的数据
            for tech in tech_cols:
                if tech not in df_investment.columns:
                    continue
                
                # 确保技术在省份字典中
                if tech not in result[province_key]:
                    result[province_key][tech] = {}
                
                # 获取该技术在当前年份的值
                try:
                    val = row[tech]
                    if pd.notna(val) and val != '' and val != tech:  # 确保值不是技术名称（在年份行中技术列包含技术名称）
                        # 尝试转换为数值类型
                        try:
                            # 如果是字符串，尝试转换为浮点数
                            if isinstance(val, str):
                                if val.replace('.', '', 1).isdigit():  # 检查是否是数字字符串
                                    val = float(val)
                                else:
                                    continue  # 跳过非数字字符串
                            
                            result[province_key][tech][str(year)] = float(val)
                        except (ValueError, TypeError) as e:
                            print(f"警告: 无法将 '{val}' 转换为浮点数。省份:{province_key}, 技术:{tech}, 年份:{year}")
                except Exception as e:
                    print(f"处理 {province_key} 的 {tech} 在 {year} 年的数据时出错: {e}")
    
    # 检查结果，确保所有省份和技术都有数据
    empty_provinces = []
    for province, tech_data in result.items():
        if not any(year_data for tech, year_data in tech_data.items()):
            empty_provinces.append(province)
    
    if empty_provinces:
        print(f"警告: 以下省份没有数据: {empty_provinces}")
    
    # 输出示例数据（前两个省份的部分数据）
    print("\n数据解析结果示例:")
    sample_provinces = list(result.keys())[:2]
    for province in sample_provinces:
        print(f"省份: {province}")
        for tech in list(result[province].keys())[:3]:
            print(f"  {tech}: {result[province][tech]}")
    
    # 输出JSON文件
    output_path = os.path.join(os.path.dirname(file_path), "json", "elc_investment.json")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"\n数据已成功导出到 {output_path}")
    
except FileNotFoundError:
    print(f"未找到文件 'source data 对接版.xlsx'，请确保文件在当前路径下。")
except Exception as e:
    import traceback
    print(f"处理电力投资数据时发生错误: {e}")
    traceback.print_exc()


==== 电力投资数据解析与导出 ====

数据解析结果示例:
省份: BEIJ
  coal: {'2025': 0.77, '2030': 0.77, '2035': 0.77, '2040': 0.77, '2045': 0.77, '2050': 0.77, '2055': 0.77, '2060': 0.77}
  oil: {'2025': 0.22, '2030': 0.22, '2035': 0.22, '2040': 0.22, '2045': 0.22, '2050': 0.22, '2055': 0.22, '2060': 0.22}
  gas: {'2025': 10.0, '2030': 10.0, '2035': 10.0, '2040': 10.0, '2045': 10.0, '2050': 10.0, '2055': 10.0, '2060': 10.0}
省份: TIAN
  coal: {'2025': 12.3, '2030': 12.3, '2035': 12.3, '2040': 12.3, '2045': 12.3, '2050': 12.3, '2055': 12.3, '2060': 12.3}
  oil: {'2025': 0.0, '2030': 0.0, '2035': 0.0, '2040': 0.0, '2045': 0.0, '2050': 0.0, '2055': 0.0, '2060': 0.0}
  gas: {'2025': 3.82, '2030': 3.82, '2035': 3.82, '2040': 3.82, '2045': 3.82, '2050': 3.82, '2055': 3.82, '2060': 3.82}

数据已成功导出到 /Users/kevinding/Desktop/Github/energy-supply-v2/data/excel/json/elc_investment.json


上面的文件报废，使用新的文件

In [5]:
import pandas as pd
import json
import numpy as np

try:
    # 定义文件路径
    inv_file = 'capnew_inv.xlsx'
    
    # 读取Excel文件
    inv_df = pd.read_excel(inv_file)
    
    # 创建数据结构
    inv_data = {}
    
    # 遍历DataFrame处理数据
    current_year = None
    
    for _, row in inv_df.iterrows():
        # 获取年份和省份
        if not pd.isna(row['Unnamed: 0']):
            current_year = str(int(row['Unnamed: 0']))
        
        province = row['Unnamed: 1']
        
        # 确保省份存在于数据结构中
        if province not in inv_data:
            inv_data[province] = {
                'coal': {}, 'coal ccs': {}, 'oil': {}, 'gas': {}, 'gas ccs': {},
                'nuclear': {}, 'hydro': {}, 'biomass': {}, 'biomass ccs': {}, 
                'co-firing beccs': {}, 'wind': {}, 'pv': {}
            }
        
        # 添加各种电力技术数据
        for tech in ['coal', 'coal ccs', 'oil', 'gas', 'gas ccs', 'nuclear', 'hydro', 
                     'biomass', 'biomass ccs', 'co-firing beccs', 'wind', 'pv']:
            if not pd.isna(row[tech]):
                inv_data[province][tech][current_year] = round(float(row[tech]), 3)
    
    # 确保json目录存在
    import os
    if not os.path.exists('json'):
        os.makedirs('json')
    
    # 将数据保存到 json/inv.json 文件
    with open('json/inv.json', 'w', encoding='utf-8') as f:
        json.dump(inv_data, f, ensure_ascii=False, indent=4)
    
    print("\n电力投资数据已成功保存到 json/inv.json 文件")
    
except FileNotFoundError:
    print(f"未找到文件 '{inv_file}'，请确保文件在当前目录下。")
except Exception as e:
    print(f"处理电力投资数据时发生错误: {e}")



电力投资数据已成功保存到 json/inv.json 文件
